# This is the clean version of Test_Work.ipynb
It contains the same code, but with only the core components of the generator 

## Modules Used

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import random
import duckdb as ddb

# Further Analysis
import matplotlib.pyplot as plt
# import plotly.express as px

## Sales Invoices

In [ ]:
def generate_sales_data(num_records, start_date, end_date, regions_df, products_df):
    # Lists to store generated data
    order_dates = []
    shipment_dates = []
    regions = []
    countries = []
    product_types = []
    unit_prices = []
    unit_costs = []
    quantities = []

    # Generate data for each record
    for _ in range(num_records):
        # Randomly select region and corresponding country
        region_row = regions_df.sample().iloc[0]
        regions.append(region_row['StoreID'])
        countries.append(region_row['StoreName'])

        # Randomly select product and corresponding unit price and cost
        product_row = products_df.sample().iloc[0]
        product_types.append(product_row['Product'])
        unit_prices.append(product_row['Unit Price'])
        unit_costs.append(product_row['Unit Cost'])

        # Random order date between start and end date
        random_date = start_date + timedelta(days=np.random.randint(0, (end_date-start_date).days))
        order_dates.append(random_date)
        # Ship date between 1 and 50 days after order date
        shipment_dates.append(random_date + timedelta(days=np.random.randint(1, 51)))

        # Random quantity between 1000 and 10000
        quantities.append(np.random.randint(1000, 10001))

    # Create dataframe
    sales_df = pd.DataFrame({
        'Order_Date': order_dates,
        'Shipment_Date': shipment_dates,
        'StoreID': regions,
        'StoreName': countries,
        'Product_Type': product_types,
        'Unit_Price': unit_prices,
        'Unit_Cost': unit_costs,
        'Quantity': quantities,
        'Revenue': np.array(unit_prices) * np.array(quantities),
        'Total_Cost': np.array(unit_costs) * np.array(quantities)
    })

    sales_df['Total Profit'] = sales_df['Revenue'] - sales_df['Total_Cost']

    return sales_df

# Example usage:
regions_data = {
    'StoreID': [1,2,3,4,5,6,7,8,9,10],
    'StoreName': ["StoreNY","StoreLA","StorePA","StoreKY","StoreWA","StoreDC","StoreFL","StoreAL","StoreTX","StoreMN",]
}
regions_df = pd.DataFrame(regions_data)

products_data = {
    'Product': ["Office Supplies","Vegetables","Fruits","Cosmetics","Cereal","Baby Food","Beverages","Snacks","Clothes","Household","Personal Care","Meat"],
    'Unit Price': [651.21,154.06,9.33,437.2,205.7,255.28,47.45,152.58,109.28,668.27,81.73,421.89],
    'Unit Cost': [524.96,90.93,6.92,263.33,117.11,159.42,31.79,97.44,35.84,502.54,56.67,364.69]
}

products_df = pd.DataFrame(products_data)

start_date = pd.Timestamp('2020-01-01')
end_date = pd.Timestamp('2022-01-01')
num_records = 1000

sales_df = generate_sales_data(num_records, start_date, end_date, regions_df, products_df)


## Purchase Orders

In [ ]:
sales_data = sales_df
sales_data = sales_data.sort_values(by=['Order_Date'])


# Define product types
product_types = ['Cereal', 'Snacks', 'Beverages', 'Baby Food', 'Meat', 'Fruits', 'Vegetables', 'Personal Care', 'Cosmetics', 'Household', 'Office Supplies', 'Clothes']

# Initialize empty dataframes for purchase orders and inventory
purchase_orders = pd.DataFrame(columns=['OrderID', 'Product', 'Vendor', 'Quantity', 'OrderDate'])
Inventory_dict = {'Cereal':0, 'Snacks':0, 'Beverages':0, 'Baby Food':0, 'Meat':0, 'Fruits':0, 'Vegetables':0, 'Personal Care':0, 'Cosmetics':0, 'Household':0, 'Office Supplies':0, 'Clothes':0}
Vendor_dict = {'Cereal':'Foodco', 'Snacks':'Foodco', 'Beverages':'Foodco', 'Baby Food':'Foodco', 'Meat':'Farmco', 'Fruits':'Farmco', 'Vegetables':'Farmco', 'Personal Care':'Beautyco', 'Cosmetics':'Beautyco', 'Household':'Homeco', 'Office Supplies':'Homeco', 'Clothes':'Fashionco'}

# Purchase Order lists
Product_Ordered = []
Quantity_Purchased = []
Date_Purchased = []
Vendor = []


# Define a function to generate purchase orders

for index, row in sales_data.iterrows():
  x = 0
  product = row['Product_Type']
  quantity_sold = row['Quantity']
  order_date = row['Order_Date']
  Unit_Cost = row['Unit_Cost']

  if product in Inventory_dict.keys():
   if quantity_sold > Inventory_dict[product]:
    purchase_quantity = (quantity_sold - Inventory_dict[product]) * 1.05
    Inventory_dict[product] = (purchase_quantity + Inventory_dict[product]) - quantity_sold
    Product_Ordered.append(product)
    Quantity_Purchased.append(purchase_quantity)
    Date_Purchased.append(order_date)
    Ven = Vendor_dict[product]
    Vendor.append(Ven)
   else:
    Inventory_dict[product] = Inventory_dict[product] - quantity_sold

PO_df = pd.DataFrame(
    {'Vendor': Vendor,
     'Quantity': Quantity_Purchased,
     'Product': Product_Ordered,
     'Order_Date': Date_Purchased,
    })

Qry_PO ='''
    SELECT Vendor,
    Product,
    SUM(Quantity),
    YEAR(Order_Date) AS Year,
    QUARTER(Order_Date) AS Quarter,
    CAST(
      CASE
        WHEN QUARTER(Order_Date) = 1
          THEN CONCAT(YEAR(Order_Date), '-01-01')
        WHEN QUARTER(Order_Date) = 2
          THEN CONCAT(YEAR(Order_Date), '-04-01')
        WHEN QUARTER(Order_Date) = 3
          THEN CONCAT(YEAR(Order_Date), '-07-01')
        WHEN QUARTER(Order_Date) = 4
          THEN CONCAT(YEAR(Order_Date), '-10-01')
      END AS DATE) AS Purchase_Date
    FROM PO_df
    GROUP BY Vendor, Product, YEAR(Order_Date), QUARTER(Order_Date)
    ORDER BY Year, Quarter
'''

df_PO = ddb.sql(Qry_PO).df()

df_PO = df_PO.merge(products_df[['Product','Unit Cost']], on ='Product', how = 'left')

## Inventory

In [ ]:
Inv_DEC = pd.DataFrame({"Date" : sales_df['Shipment_Date'],"Product":sales_df["Product_Type"],"Quantity":sales_df["Quantity"],"Unit_Cost":sales_df["Unit_Cost"]})
Inv_INC = pd.DataFrame({"Date": df_PO['Purchase_Date'],"Product":df_PO['Product'],"Quantity":df_PO['sum(Quantity)'],"Unit_Cost":df_PO["Unit Cost"],"Vendor":df_PO['Vendor']})

df_INV = Inv_DEC.append(Inv_INC,ignore_index=True)

## Total Revenue Calculations

In [ ]:
# Total Revenue
qry_Trev = '''
    SELECT YEAR(Shipment_Date) AS Year, SUM(Revenue) AS Revenue
    FROM sales_df
    GROUP BY Year
    Order By Year
'''

# Total Revenue Per Product Per year
qry_rev = '''
    SELECT YEAR(Shipment_Date) AS Year, Product_Type AS Product, SUM(Revenue) AS Revenue, Unit_Cost
    FROM sales_df
    GROUP BY Year, Product, Unit_Cost
    Order By Year
'''

df_Trev = ddb.sql(qry_Trev).df()
df_rev = ddb.sql(qry_rev).df()

# Rev_dict = dict(zip(df_Trev['Year'],zip(df_Trev['Revenue'],df_Trev['Unit_Cost']))
Rev_dict = dict(zip(df_Trev['Year'],df_Trev['Revenue']))

## Scenario 1

### COGS

In [ ]:
COGS = '''
  SELECT
    YEAR(Date) as Year,
    CONCAT(YEAR(Date),'_',Quarter(Date)) AS Yr_Q,
    Product,
    SUM((Quantity * Unit_Cost)) AS Value
  FROM df_INV
  WHERE Vendor IS NULL
  GROUP BY Product, Yr_Q, Year
  ORDER BY Yr_Q
'''

COGS_df = ddb.sql(COGS).df()

# Gross Margin
#   Yearly Rev - Yearly COGS
# Yearly COGS

Yr_COGS_qry = '''
    SELECT 
        Year,
        Sum(Value) AS Value
    FROM COGS_df
    GROUP BY Year
'''
COGSyr_df = ddb.sql(Yr_COGS_qry).df()
COGS_dict = dict(zip(COGSyr_df['Year'],COGSyr_df['Value']))

PRC_dict = {}
PM_dict = {}
# Substract COGS from Rev each year 
for Year in Rev_dict.keys():
    PRC_dict[Year] = {"Rev":Rev_dict[Year],"COGS":COGS_dict[Year],"PM":Rev_dict[Year]-COGS_dict[Year]}
    PM_dict[Year] = Rev_dict[Year]-COGS_dict[Year]
PM_Rev_df = pd.DataFrame({'PM':PM_dict,'REV':Rev_dict})

In [ ]:
# Important Graph showing the relationship between the Rev, PM, and COGS

PR_df2 = PM_Rev_df[PM_Rev_df.index != 2022]
PR_df2['dif'] = PR_df2['REV'] - PR_df2['PM']

x = PR_df2.index
y1 = PR_df2['PM']
y2 = PR_df2['REV']
y3 = PR_df2['dif']

plt.plot(x, y1, label='PM')
plt.plot(x, y2, label='Rev')
plt.legend(loc='upper left')
ax2 = plt.gca().twinx()
ax2.plot(x, y3, label='Dif', color='r')
ax2.legend(loc='upper right')
plt.show()


### Expenses for Scenario 1

In [ ]:
Expenses = {
    'Avg_NetProfit':(3,"Not used to calc net profit"),
    'Rent': (6,10),
    'Insurance': (2,4),
    'Wages': (15,20),
    'Ads': (6,8),
    'Taxes': (13, "A fixed amount")
}

Exp_dict = {}
Exp_list = []

for i, year in enumerate(PRC_dict.keys()):
    gross = PRC_dict[year]["PM"]
    Rev = PRC_dict[year]["Rev"]
    COGS = PRC_dict[year]["COGS"]
    Year = year

    
    Rent = (random.randrange(Expenses['Rent'][0],Expenses['Rent'][1])/100)*gross
    Insurance = (random.randrange(Expenses['Insurance'][0],Expenses['Insurance'][1])/100)*gross
    Wages = (random.randrange(Expenses['Wages'][0],Expenses['Wages'][1])/100)*gross
    Ads = (random.randrange(Expenses['Wages'][0],Expenses['Wages'][1])/100)*gross
    Net = gross - (Rent+Insurance+Wages+Ads)
    EBIT = (1 - (Expenses['Taxes'][0]/100)) * Net
    Tax = (Expenses['Taxes'][0]/100) * Net
    Net_In = EBIT - Tax

    Exp_dict = {'Year':Year,'Rev':Rev,'COGS':COGS,'Gross_Profit':gross,'Rent':Rent,'Insurance':Insurance,'Wages':Wages,'Ads':Ads,'Net_Profit':Net,'EBIT': EBIT, 'Taxes':Tax, 'Net_Income': Net_In}
    Exp_list.append(Exp_dict)


Exp_df = pd.DataFrame(Exp_list)

print(Exp_df.info())
    



